# HW-5 CART
Wyatt Blair

SID: 10420296

3/10/24


In [1]:
import pandas as pd
import numpy as np

## 5.1 -- Salary

In [16]:
occupation = ['Service'] * 3 + ['Management'] * 4 + ['Sales'] * 2 + ['Staff'] * 2
gender = ['F', 'M', 'M',   'M', 'F', 'M', 'F',  'F', 'M',  'F', 'M']
age = [45, 25, 33, 25, 35, 26, 45, 40, 30, 50, 25]
salary = np.array([48, 25, 35, 45, 65, 45, 70, 50, 40, 40, 25]) * 1e3

In [18]:
values = list(zip(
    occupation,   gender,    age,   salary
))
columns = [
    'occupation', 'gender', 'age', 'salary'
]
df = pd.DataFrame(data=values, columns=columns); df

,occupation,gender,age,salary
0,Service,F,45,48000.0
1,Service,M,25,25000.0
2,Service,M,33,35000.0
3,Management,M,25,45000.0
4,Management,F,35,65000.0
5,Management,M,26,45000.0
6,Management,F,45,70000.0
7,Sales,F,40,50000.0
8,Sales,M,30,40000.0
9,Staff,F,50,40000.0


In [19]:
features = columns.copy(); features.remove('salary')

In [20]:
def age_splits(age):

    if age <= 30:
        return "0-30"
    elif age <= 40:
        return "31-40"
    else:
        return "Above 40"

def salary_splits(salary):

    salary /= 1e3

    if salary <= 35:
        return '1'
    elif 35 < salary <= 45:
        return '2'
    elif 45 < salary <= 55:
        return '3'
    else:
        return '4'

In [21]:
df['age_splits'] = df['age'].apply(age_splits)
df['salary_splits'] = df['salary'].apply(salary_splits)


In [22]:
df

,occupation,gender,age,salary,age_splits,salary_splits
0,Service,F,45,48000.0,Above 40,3
1,Service,M,25,25000.0,0-30,1
2,Service,M,33,35000.0,31-40,1
3,Management,M,25,45000.0,0-30,2
4,Management,F,35,65000.0,31-40,4
5,Management,M,26,45000.0,0-30,2
6,Management,F,45,70000.0,Above 40,4
7,Sales,F,40,50000.0,31-40,3
8,Sales,M,30,40000.0,0-30,2
9,Staff,F,50,40000.0,Above 40,2
